In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import os
import pandas as pd

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d sergioagudelo/phishing-url-detection
!ls

Dataset URL: https://www.kaggle.com/datasets/sergioagudelo/phishing-url-detection
License(s): MIT
phishing-url-detection.zip: Skipping, found more recently modified local copy (use --force to force download)
kaggle.json  out.csv  phishing-url-detection.zip  sample_data


In [ ]:
!unzip phishing-url-detection.zip

Archive:  phishing-url-detection.zip
replace out.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: out.csv                 


In [ ]:
import pandas as pd

df = pd.read_csv('out.csv')
print(df.head())

                                             url          source       label  \
0  apaceast.cloudguest.central.arubanetworks.com  Cisco-Umbrella  legitimate   
1                              quintadonoval.com        Majestic  legitimate   
2                               nomadfactory.com        Majestic  legitimate   
3                               tvarenasport.com        Majestic  legitimate   
4                    widget.cluster.groovehq.com  Cisco-Umbrella  legitimate   

   url_length  starts_with_ip  url_entropy  has_punycode  digit_letter_ratio  \
0          45           False     3.924535         False                 0.0   
1          17           False     3.572469         False                 0.0   
2          16           False     3.327820         False                 0.0   
3          16           False     3.500000         False                 0.0   
4          27           False     3.930270         False                 0.0   

   dot_count  at_count  dash_count  tl